In [ ]:
import pandas as pd
df = pd.read_csv("movie_sample.csv")
df.head()

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl


In [ ]:
count_univ = np.arange(0, df['rating_count'].max()+1, 1)
rating_univ = np.arange(0, 5.1, 0.1)
reco_univ = np.arange(0, 101, 1)

avg_rating = ctrl.Antecedent(rating_univ, 'avg_rating')
rating_count = ctrl.Antecedent(count_univ, 'rating_count')
recommendation = ctrl.Consequent(reco_univ, 'recommendation')

avg_rating['low'] = fuzz.trimf(rating_univ, [0, 0, 3])
avg_rating['medium'] = fuzz.trimf(rating_univ, [2.5, 3.5, 4.5])
avg_rating['high'] = fuzz.trimf(rating_univ, [3.5, 5, 5])

rating_count['few'] = fuzz.trimf(count_univ, [0, 0, 50])
rating_count['moderate'] = fuzz.trimf(count_univ, [30, 100, 200])
rating_count['many'] = fuzz.trimf(count_univ, [150, 300, 300])

recommendation['low'] = fuzz.trimf(reco_univ, [0, 0, 50])
recommendation['medium'] = fuzz.trimf(reco_univ, [30, 50, 70])
recommendation['high'] = fuzz.trimf(reco_univ, [60, 100, 100])


In [ ]:
rule1 = ctrl.Rule(avg_rating['high'] & rating_count['many'], recommendation['high'])
rule2 = ctrl.Rule(avg_rating['medium'] & rating_count['many'], recommendation['medium'])
rule3 = ctrl.Rule(avg_rating['low'] | rating_count['few'], recommendation['low'])

reco_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
reco_sim = ctrl.ControlSystemSimulation(reco_ctrl)


In [ ]:
scores = []
for _, row in df.iterrows():
    reco_sim.input['avg_rating'] = float(row['avg_rating'])
    reco_sim.input['rating_count'] = float(row['rating_count'])
    reco_sim.compute()
    scores.append(reco_sim.output['recommendation'])

df['recommendation_score'] = scores
df['recommendation_label'] = pd.cut(df['recommendation_score'], 
                                    bins=[0, 40, 70, 100], 
                                    labels=['Not Recommended','Maybe','Recommended'])
df.to_csv("movie_sample_with_reco.csv", index=False)
df.head()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6,4))
sns.histplot(df['avg_rating'])
plt.title("Distribusi avg_rating")
plt.savefig("dist_avg_rating.png")

plt.figure(figsize=(6,4))
sns.histplot(df['rating_count'])
plt.title("Distribusi rating_count")
plt.savefig("dist_rating_count.png")

plt.figure(figsize=(6,4))
sns.histplot(df['recommendation_score'])
plt.title("Distribusi recommendation_score")
plt.savefig("dist_recommendation_score.png")

plt.show()
